In [25]:
import pandas as pd
import nbconvert
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
from matplotlib import cm
import ipywidgets as widgets
from IPython.display import display

In [26]:
import ipywidgets as widgets
from IPython.display import display

# Define the growth_rates dictionary
growth_rates = {
    'robert_income': 0.03,
    'isabel_income': 0.03,
    'expenses': 0.03,
    'assets': 0.05
}

# Define the base values dictionary
base_values = pd.DataFrame({
        'year': [2023],
        'robert_income': [250_000],
        'isabel_income': [150_000],
        'expenses': [120_000],
        'assets': [800_000]
}).set_index('year')

# Create widgets for each variable in growth_rates
growth_rates_widgets = {}
for variable, default_value in growth_rates.items():
    # Customize the widget appearance
    widget = widgets.FloatText(
        value=default_value,
        description=variable.replace('_', ' ').title() + ' growth rate',  # Change variable name formatting
        step=0.01,
        layout=widgets.Layout(width='300px'),  # Set widget width
        style={'description_width': 'initial'}  # Prevent description truncation
    )
    growth_rates_widgets[variable] = widget

# Create widgets for income, expenses, assets initial values
base_values_widgets = {}
for variable, default_value in base_values.items():
    # Customize the widget appearance
    widget = widgets.FloatText(
        value=default_value,
        description=variable.replace('_', ' ').title(),  # Change variable name formatting
        step=5_000,
        layout=widgets.Layout(width='300px'),  # Set widget width
        style={'description_width': 'initial'}  # Prevent description truncation
    )
    base_values_widgets[variable] = widget


# Display the widgets
for widget in base_values_widgets.values():
    display(widget)
    
for widget in growth_rates_widgets.values():
    display(widget)

/home/deck/Documents/Github/financial-projections/.venv/lib/python3.10/site-packages/traitlets/traitlets.py:2438: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



FloatText(value=250000.0, description='Robert Income', layout=Layout(width='300px'), step=5000.0, style=Descri…

FloatText(value=150000.0, description='Isabel Income', layout=Layout(width='300px'), step=5000.0, style=Descri…

FloatText(value=120000.0, description='Expenses', layout=Layout(width='300px'), step=5000.0, style=Description…

FloatText(value=800000.0, description='Assets', layout=Layout(width='300px'), step=5000.0, style=DescriptionSt…

FloatText(value=0.03, description='Robert Income growth rate', layout=Layout(width='300px'), step=0.01, style=…

FloatText(value=0.03, description='Isabel Income growth rate', layout=Layout(width='300px'), step=0.01, style=…

FloatText(value=0.03, description='Expenses growth rate', layout=Layout(width='300px'), step=0.01, style=Descr…

FloatText(value=0.05, description='Assets growth rate', layout=Layout(width='300px'), step=0.01, style=Descrip…

In [59]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Define the initial shocks DataFrame
shocks = pd.DataFrame(columns=['Year', 'Shock Type', 'Amount', 'Comment'])

# Create input widgets for shock details
year_widget = widgets.IntText(
    value=2023,  # Set default value
    description='Year:',
    layout=widgets.Layout(width='300px')
)

shock_type_widget = widgets.Dropdown(
    options=['robert_income', 'isabel_income', 'expenses', 'assets'],
    value='robert_income',
    description='Shock Type:',
    layout=widgets.Layout(width='300px')
)

amount_widget = widgets.FloatText(
    description='Amount:',
    layout=widgets.Layout(width='300px')
)

comment_widget = widgets.Text(
    description='Comment:',
    layout=widgets.Layout(width='300px')
)

add_shock_button = widgets.Button(
    description='Add Shock',
    layout=widgets.Layout(width='150px')
)

# Create output widget for shocks
shocks_output = widgets.Output()

def add_shock(button):
    year = year_widget.value
    shock_type = shock_type_widget.value
    amount = amount_widget.value
    comment = comment_widget.value
    
    shocks.loc[len(shocks)] = [year, shock_type, amount, comment]
    
    # Reset the input fields
    year_widget.value = 2023  # Reset the year value
    shock_type_widget.value = 'robert_income'
    amount_widget.value = 0
    comment_widget.value = ""

# Attach the event handler to the button
add_shock_button.on_click(add_shock)

# Display the input widgets and output
display(year_widget, shock_type_widget, amount_widget, comment_widget, add_shock_button)

def display_shocks():
    with shocks_output:
        clear_output()
        display(HTML('<h4>Current Shocks:</h4>'))
        display(shocks)

# Display the initial shocks
display_shocks()


IntText(value=2023, description='Year:', layout=Layout(width='300px'))

Dropdown(description='Shock Type:', layout=Layout(width='300px'), options=('robert_income', 'isabel_income', '…

FloatText(value=0.0, description='Amount:', layout=Layout(width='300px'))

Text(value='', description='Comment:', layout=Layout(width='300px'))

Button(description='Add Shock', layout=Layout(width='150px'), style=ButtonStyle())

Output()

In [58]:
type(shocks_output)

ipywidgets.widgets.widget_output.Output

In [11]:


def financial_projection(robert_income, isabel_income, expenses, assets, growth_rates, shocks, volatility, simulations):
    # set defaults for charts
    pio.templates.default = "plotly_white"

    @np.vectorize
    def calculate_tax(income):
        brackets = [9950, 40525, 86375, 164925, 209425, 523600]
        rates = [0.10, 0.12, 0.22, 0.24, 0.32, 0.35, 0.37]
        tax = 0
        for i in range(len(brackets)):
            if income > brackets[i]:
                if i == 0:
                    tax += rates[i] * brackets[i]
                else:
                    tax += rates[i] * (brackets[i] - brackets[i - 1])
            else:
                if i == 0:
                    tax += rates[i] * income
                else:
                    tax += rates[i] * (income - brackets[i - 1])
                break
        if income > brackets[-1]:
            tax += rates[-1] * (income - brackets[-1])

        return tax

    # parameters
    variables = ['robert_income', 'isabel_income', 'expenses', 'assets']

    # create initial setup DataFrame
    data = pd.DataFrame({
        'year': [2023],
        'robert_income': [robert_income],
        'isabel_income': [isabel_income],
        'expenses': [expenses],
        'assets': [assets]
    }).set_index('year')

    growth_assumptions = growth_rates

    volatility = volatility  # standard deviation of asset growth
    simulations = simulations  # number of simulations

    # create a DataFrame to hold the future projections
    projection = pd.DataFrame(index=range(2023, 2083))

    # initialize a DataFrame with simulations for assets
    asset_simulations = pd.DataFrame(1 + volatility * np.random.standard_normal(size=(60, simulations)),
                                     index=projection.index,
                                     columns=['simulation_' + str(i) for i in range(simulations)]
    )

    # chain all
    asset_simulations = asset_simulations.cumprod()


    # loop over years
    for year in projection.index:
        if year == 2023:
            # handle base year
            for var in variables:
                projection.loc[year, var] = data.loc[2023, var]
                asset_simulations.loc[year] = data.loc[2023, 'assets']
        else:
            # apply growth assumptions and shocks
            for var in variables:
                projection.loc[year, var] = projection.loc[year - 1, var] * (1 + growth_assumptions[var])
                if year in shocks and var in shocks[year]:
                    shock, _ = shocks[year][var]
                    projection.loc[year, var] += shock
            
            # calculate household income and savings
            projection.loc[year, 'household_income'] = projection.loc[year, 'robert_income'] + projection.loc[year, 'isabel_income']
            projection.loc[year, 'taxes'] = calculate_tax(projection.loc[year, 'household_income'])
            projection.loc[year, 'net_household_income'] = projection.loc[year, 'household_income'] - projection.loc[year, 'taxes']

            # calculate savings
            projection.loc[year, 'savings'] = projection.loc[year, 'net_household_income'] - projection.loc[year, 'expenses']

            # add savings to assets
            projection.loc[year, 'assets'] += projection.loc[year, 'savings']

            # add volatility to assets
            asset_simulations.loc[year] = projection.loc[year - 1, 'assets'] * (asset_simulations.loc[year])


    
    # plot income, expenses, and savings
    fig = go.Figure(layout=go.Layout(template='plotly_white'))

    for var in ['robert_income', 'isabel_income', 'expenses', 'savings', 'household_income','net_household_income','taxes']:
        fig.add_trace(go.Scatter(
            x=projection.index,
            y=projection[var], 
            mode='lines', 
            name=var.replace('_',' ').title()))

    fig.show()

    # plot asset simulations as a fan chart
    fig = go.Figure()

    percentiles = [1, 5, 20, 50, 80, 95, 99]
    colors = [cm.Blues(x) for x in np.linspace(0.01, 1, 7)]

    for i in range(len(percentiles)):
        percentile = percentiles[i]
        color = colors[i]
        asset_percentile = asset_simulations.apply(lambda x: np.percentile(x, percentile), axis=1)
        fig.add_trace(go.Scatter(
            x=asset_percentile.index, 
            y=asset_percentile, 
            fill='tonexty', 
            fillcolor='rgba'+str(color),
            line_color='rgba'+str(color),
            name=str(percentile)+'th percentile'))

    fig.show()


    # plot shocks
    all_shock_values = []

    for shock_type in ['assets', 'robert_income', 'isabel_income', 'expenses']:
        for year, shocks_in_year in shocks.items():
            if shock_type in shocks_in_year:
                all_shock_values.append(shocks_in_year[shock_type][0])

    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, shared_yaxes='rows')

    for shock_type, subplot in zip(['assets', 'robert_income', 'isabel_income', 'expenses'], [1, 2, 3, 4]):
        shock_years = []
        shock_values = []
        hover_texts = []  # New list to store hover text labels
        for year, shocks_in_year in shocks.items():
            if shock_type in shocks_in_year:
                shock_years.append(year)
                shock_values.append(shocks_in_year[shock_type][0])
                hover_texts.append(shocks_in_year[shock_type][1])  # Add the hover text label to the list
        fig.add_trace(go.Bar(
            x=shock_years, 
            y=shock_values, 
            name=shock_type.replace('_',' ').title() + ' Shocks', 
            text=hover_texts, 
            textposition='outside', 
            hovertemplate='%{text}', 
            textfont=dict(color='rgba(0,0,0,0)')), 
            row=subplot, 
            col=1)

    fig.update_xaxes(range=[2023, 2082])
    fig.update_yaxes(range=[min(all_shock_values), max(all_shock_values)])
    fig.update_layout(template='plotly_white')
    fig.show()

# Example usage
financial_projection(100000, 200000, 50000, 800000, growth_rates, shocks, 0.08, 1000)


NameError: name 'shocks' is not defined